In [25]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np 


In [26]:
## load the trained model ,scaler pickle model 
model = load_model('model.h5')
## load th encoder and scaler 
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [27]:
## Example input data 
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [28]:
## one hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/sandipmahata/Documents/my self/krish naik  course /End ot End deeplearning project using ANN/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [29]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [30]:
## encoded categorical varicble 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [31]:
## concatintion one hot encoded
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:
## scaling input data 
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [33]:
## predict churn 
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 68ms/step


array([[0.04242602]], dtype=float32)

In [35]:
prediction_proba = prediction[0][0]
prediction_proba

0.042426016

In [36]:
if prediction_proba >0.5:
    print("the customer is likely to churn")
else :
    print("the customer is not likely to churn")

the customer is not likely to churn
